In [16]:
import pandas as pd 
import numpy as np
import os
import requests
import time
from datetime import date
from bs4 import BeautifulSoup

## Read Data

In [2]:
os.path.abspath('')

'/Users/liamisaacs/Desktop/Personal Github Repositories/metis-project3/v0.0.0'

In [3]:
search_params = 'all'
fldr = os.path.join(os.path.abspath(''), '..', 'data')

for f in os.listdir(fldr):
    
    if search_params in f:
        
        print(f)
        fp = os.path.join(fldr, f)

fp

api-all_20210925.csv


'/Users/liamisaacs/Desktop/Personal Github Repositories/metis-project3/v0.0.0/../data/api-all_20210925.csv'

In [4]:
df = pd.read_csv(fp)

/Users/liamisaacs/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Helpers

In [5]:
#No header/user-agent cycling?
headers = {
    'sec-ch-ua': '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
    'Referer': '',
    'sec-ch-ua-mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.192 Safari/537.36'
}

In [7]:
def accessor_id(user_name: 'str') -> 'int': 
    
    """ Get accesssor summoner ID to feed into OPGG API """
    
    def html_safe(user_name: "str") -> "str": 
        
        """Return HTML safe username to search"""

        html_str = user_name
        html_str.replace(' ', '+')

        return html_str
        
    
    if not isinstance(user_name, str): 
        return;
    
    base_url = 'https://na.op.gg/summoner/'
    url_substring = 'userName={}'.format(html_safe(user_name)) 
    url = base_url + url_substring
    
    try:
        r = requests.get(url, headers=headers)
        #print(f'Fetching data for {user_name} @ {url} ...')
            
        soup = BeautifulSoup(r.text, 'html.parser')
        
        #OPGG accesssor ID (for anything involving API requests)
        divs = soup.findAll("div", {"data-summoner-id" : True})
        ids = set(
                    [x['data-summoner-id'] for x in divs]
                 )
    
        #Check uniqueness
        if len(ids) != 1:
            #print(f'Multiple or no IDs found, ids: {ids} for user {user_name}\nMost likely due to name change        
            return 
        
        
        accesssor_id = ids.pop()        
        return accesssor_id
    
    except:
        print(f'Connection error for {user_name} @ {url}')

## Scrape OPgg

In [8]:
assert df.shape[0] == len(set(df.summonerName.values)), 'Oh no there are duplicate names...'

In [9]:
def print_progress(iteration, total, prefix ='', suffix ='', decimals =1, length =100, fill ='█', printEnd="\r"):
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end =printEnd)

    if iteration == total:
        print()

In [9]:
#Runtime: 5 hrs.
start: 8:51 pm
note: do not turn off computer

SyntaxError: invalid syntax (<ipython-input-9-706eb9845d2c>, line 2)

In [10]:
main = pd.DataFrame({'user_name': '', 'user_id': ''}, index=[0])

print_progress(0, len(df.summonerName.values), prefix ='', suffix ='Complete')

for i,name in enumerate(df.summonerName.values):
    
    user_id = accessor_id(name)
    
    if user_id:
    
        tmp = pd.DataFrame({'user_name': name, 'user_id': user_id}, index=[0])

        main = main.append(tmp)
    
        if i%1000==0: 
            print("Scraped {} of {}, {} remaining".format(i,len(df.summonerName.values),len(df.summonerName.values)-i))
    
    #if i%100 ==0: 
    #time.sleep(10) #Politeness is key. 100 requests/2 min. is pretty polite.
                       #Considering most scrapers run at 32/sec ...
    
    print_progress(i, len(df.summonerName.values), prefix ='', suffix ='Complete')

Scraped 0 of 43732, 43732 remaining-------------------------------------------------------------------| 0.0% Complete
Scraped 1000 of 43732, 42732 remaining----------------------------------------------------------------| 2.3% Complete
Scraped 2000 of 43732, 41732 remaining----------------------------------------------------------------| 4.6% Complete
Scraped 3000 of 43732, 40732 remaining----------------------------------------------------------------| 6.9% Complete
Scraped 4000 of 43732, 39732 remaining----------------------------------------------------------------| 9.1% Complete
Scraped 5000 of 43732, 38732 remaining----------------------------------------------------------------| 11.4% Complete
Scraped 6000 of 43732, 37732 remaining----------------------------------------------------------------| 13.7% Complete
Scraped 7000 of 43732, 36732 remaining----------------------------------------------------------------| 16.0% Complete
Scraped 8000 of 43732, 35732 remaining---------------

In [11]:
main

,user_name,user_id
0,,
0,Gorica,31531320
0,black player,47442009
0,Dragoon,30575802
0,c2 meteos,46125229
...,...,...
0,toastyboi2,103707273
0,LoneWuffy,100999567
0,ZeWorst01,105592409
0,gratt14,100999702


In [12]:
frame = main.copy()

In [13]:
frame.shape

(43265, 2)

In [14]:
frame.reset_index(drop=True)

,user_name,user_id
0,,
1,Gorica,31531320
2,black player,47442009
3,Dragoon,30575802
4,c2 meteos,46125229
...,...,...
43260,toastyboi2,103707273
43261,LoneWuffy,100999567
43262,ZeWorst01,105592409
43263,gratt14,100999702


In [19]:
# setting up filepath 
data_folder = ''
base_filename = 's2-user_ids'
timestamp = str(date.today()).replace('-','')
ext = '.csv'
filepath = '_'.join([data_folder+base_filename,timestamp+ext])

#export
frame.to_csv(filepath,index=False,header=True)